In [2]:
!wget http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531887/train_sample.zip

--2021-04-11 12:39:16--  http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531887/train_sample.zip
Resolving tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)... 118.31.232.194
Connecting to tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)|118.31.232.194|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 540689175 (516M) [application/zip]
Saving to: ‘train_sample.zip.1’

train_sample.zip.1  100%[===================>] 515.64M  5.85MB/s    in 95s     

2021-04-11 12:40:52 (5.43 MB/s) - ‘train_sample.zip.1’ saved [540689175/540689175]



* unzip -qq 
 
 -q     perform operations quietly (-qq = even quieter).  Ordinarily unzip prints the names of the files it's extracting or testing, the extrac‐
              tion  methods,  any  file or zipfile comments that may be stored in the archive, and possibly a summary when finished with each archive.
              The -q[q] options suppress the printing of some or all of these messages.

In [3]:
!unzip -qq train_sample.zip
!\rm train_sample.zip

In [4]:
!wget http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531887/test_a.zip

--2021-04-11 12:47:46--  http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531887/test_a.zip
Resolving tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)... 118.31.232.194
Connecting to tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)|118.31.232.194|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1092637852 (1.0G) [application/zip]
Saving to: ‘test_a.zip’

test_a.zip          100%[===================>]   1.02G  6.03MB/s    in 3m 12s  

2021-04-11 12:50:59 (5.42 MB/s) - ‘test_a.zip’ saved [1092637852/1092637852]



In [5]:
!unzip -qq test_a.zip
!\rm test_a.zip

## 环境要求

- TensorFlow的版本：2.0 + 
- keras
- sklearn
- librosa

In [13]:
# 基本库

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import MinMaxScaler

## 加载深度学习框架

In [7]:
# 搭建分类模型所需要的库

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from tensorflow.keras.utils import to_categorical 

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

## 加载音频处理库

In [8]:
!pip install librosa --user

     |████████████████████████████████| 183 kB 20.4 MB/s eta 0:00:01
     |████████████████████████████████| 377 kB 43.3 MB/s eta 0:00:01
     |████████████████████████████████| 323 kB 55.1 MB/s eta 0:00:01
     |████████████████████████████████| 51 kB 87 kB/s s eta 0:00:01
  Created wheel for librosa: filename=librosa-0.8.0-py3-none-any.whl size=201376 sha256=da482419b9b0bbb2da4c099aae75713379a8f28fd39d240400b828d822c0acd7
  Stored in directory: /home/azureuser/.cache/pip/wheels/32/2c/ce/86e49d4769aceba728421c24c0d726054bf4ca01175ff42bdd
  Created wheel for audioread: filename=audioread-2.1.9-py3-none-any.whl size=23140 sha256=a71cb186d041dcd65e7ca41deb4ba9f46a0c6b72be6172f7f86a19517d386898
  Stored in directory: /home/azureuser/.cache/pip/wheels/de/14/0a/863e4ed680b3204444cf486733e609d7ff7abe8fceafab67dc
  Created wheel for resampy: filename=resampy-0.2.2-py3-none-any.whl size=320718 sha256=75e20a062455d3c4a05fde8eabe680addad1659e314838bddb18e6264b4035a7
  Stored in directory: /home/

In [9]:
# 其他库

import os
import librosa
import librosa.display
import glob 

## 特征提取以及数据集的建立

In [10]:
feature = []
label = []
# 建立类别标签，不同类别对应不同的数字。
label_dict = {'aloe': 0, 'burger': 1, 'cabbage': 2,'candied_fruits':3, 'carrots': 4, 'chips':5,
                  'chocolate': 6, 'drinks': 7, 'fries': 8, 'grapes': 9, 'gummies': 10, 'ice-cream':11,
                  'jelly': 12, 'noodles': 13, 'pickles': 14, 'pizza': 15, 'ribs': 16, 'salmon':17,
                  'soup': 18, 'wings': 19}
label_dict_inv = {v:k for k,v in label_dict.items()}

In [11]:
from tqdm import tqdm
def extract_features(parent_dir, sub_dirs, max_file=10, file_ext="*.wav"):
    c = 0
    label, feature = [], []
    for sub_dir in sub_dirs:
        for fn in tqdm(glob.glob(os.path.join(parent_dir, sub_dir, file_ext))[:max_file]): # 遍历数据集的所有文件
            
           # segment_log_specgrams, segment_labels = [], []
            #sound_clip,sr = librosa.load(fn)
            #print(fn)
            label_name = fn.split('/')[-2]
            label.extend([label_dict[label_name]])
            X, sample_rate = librosa.load(fn,res_type='kaiser_fast')
            mels = np.mean(librosa.feature.melspectrogram(y=X,sr=sample_rate).T,axis=0) # 计算梅尔频谱(mel spectrogram),并把它作为特征
            feature.extend([mels])
            
    return [feature, label]

In [14]:
# 自己更改目录
parent_dir = './train_sample/'
save_dir = "./"
folds = sub_dirs = np.array(['aloe','burger','cabbage','candied_fruits',
                             'carrots','chips','chocolate','drinks','fries',
                            'grapes','gummies','ice-cream','jelly','noodles','pickles',
                            'pizza','ribs','salmon','soup','wings'])

# 获取特征feature以及类别的label
temp = extract_features(parent_dir,sub_dirs,max_file=100)

100%|██████████| 35/35 [00:04<00:00,  7.67it/s]


In [15]:
temp = np.array(temp)
data = temp.transpose()

In [16]:
# 获取特征
X = np.vstack(data[:, 0])

# 获取标签
Y = np.array(data[:, 1])
print('X的特征尺寸是：',X.shape)
print('Y的特征尺寸是：',Y.shape)

X的特征尺寸是： (1000, 128)
Y的特征尺寸是： (1000,)


In [17]:
# 在Keras库中：to_categorical就是将类别向量转换为二进制（只有0和1）的矩阵类型表示
Y = to_categorical(Y)

In [18]:
'''最终数据'''
print(X.shape)
print(Y.shape)

(1000, 128)
(1000, 20)


In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1, stratify=Y)
print('训练集的大小',len(X_train))
print('测试集的大小',len(X_test))

训练集的大小 750
测试集的大小 250


In [20]:
X_train = X_train.reshape(-1, 16, 8, 1)
X_test = X_test.reshape(-1, 16, 8, 1)

## 建立模型

### 搭建CNN网络

In [21]:
model = Sequential()

# 输入的大小
input_dim = (16, 8, 1)

model.add(Conv2D(64, (3, 3), padding = "same", activation = "tanh", input_shape = input_dim))# 卷积层
model.add(MaxPool2D(pool_size=(2, 2)))# 最大池化
model.add(Conv2D(128, (3, 3), padding = "same", activation = "tanh")) #卷积层
model.add(MaxPool2D(pool_size=(2, 2))) # 最大池化层
model.add(Dropout(0.1))
model.add(Flatten()) # 展开
model.add(Dense(1024, activation = "tanh"))
model.add(Dense(20, activation = "softmax")) # 输出层：20个units输出20个类的概率

# 编译模型，设置损失函数，优化方法以及评价标准
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 8, 64)         640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 4, 64)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 2, 128)         0         
_________________________________________________________________
dropout (Dropout)            (None, 4, 2, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              1

In [23]:
# 训练模型
model.fit(X_train, Y_train, epochs = 20, batch_size = 15, validation_data = (X_test, Y_test))

Train on 750 samples, validate on 250 samples
Epoch 1/20
750/750 [==============================] - 19s 26ms/sample - loss: 2.8676 - accuracy: 0.1347 - val_loss: 2.6953 - val_accuracy: 0.1840
Epoch 2/20
750/750 [==============================] - 0s 369us/sample - loss: 2.5169 - accuracy: 0.2440 - val_loss: 2.6012 - val_accuracy: 0.2360
Epoch 3/20
750/750 [==============================] - 0s 370us/sample - loss: 2.2814 - accuracy: 0.3120 - val_loss: 2.4125 - val_accuracy: 0.2920
Epoch 4/20
750/750 [==============================] - 0s 365us/sample - loss: 2.0839 - accuracy: 0.3520 - val_loss: 2.4674 - val_accuracy: 0.2760
Epoch 5/20
750/750 [==============================] - 0s 394us/sample - loss: 1.8885 - accuracy: 0.4267 - val_loss: 2.4941 - val_accuracy: 0.3120
Epoch 6/20
750/750 [====================

### 预测测试集

In [24]:
def extract_features(test_dir, file_ext="*.wav"):
    feature = []
    for fn in tqdm(glob.glob(os.path.join(test_dir, file_ext))[:]): # 遍历数据集的所有文件
        X, sample_rate = librosa.load(fn,res_type='kaiser_fast')
        mels = np.mean(librosa.feature.melspectrogram(y=X,sr=sample_rate).T,axis=0) # 计算梅尔频谱(mel spectrogram),并把它作为特征
        feature.extend([mels])
    return feature

In [25]:
X_test = extract_features('./test_a/')

100%|██████████| 2000/2000 [04:30<00:00,  7.39it/s]


In [26]:
X_test = np.vstack(X_test)
predictions = model.predict(X_test.reshape(-1, 16, 8, 1))

In [27]:
preds = np.argmax(predictions, axis = 1)
preds = [label_dict_inv[x] for x in preds]

path = glob.glob('./test_a/*.wav')
result = pd.DataFrame({'name':path, 'label': preds})

result['name'] = result['name'].apply(lambda x: x.split('/')[-1])
result.to_csv('submit.csv',index=None)

In [30]:
result.describe()

,name,label
count,2000,2000
unique,2000,20
top,IK77EC2GUL.wav,pickles
freq,1,179


In [32]:
result.head()

,name,label
0,00B7EQA2MJ.wav,gummies
1,00ISO9XPIQ.wav,cabbage
2,01CF45NDOQ.wav,aloe
3,039RA8859M.wav,burger
4,03KSQR5VOP.wav,drinks


In [33]:
!ls ./test_a/*.wav | wc -l

2000


In [34]:
!wc -l submit.csv

2001 submit.csv
